<a href="https://colab.research.google.com/github/xinh3ng/ds-research/blob/colab/pysyft_federated_learning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Links

"""
!pip install scikit-learn
!pip install syft


     |████████████████████████████████| 378kB 4.5MB/s 
     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 1.8MB 47.0MB/s 
     |████████████████████████████████| 450kB 44.9MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 204kB 57.5MB/s 
     |████████████████████████████████| 2.0MB 45.7MB/s 
     |████████████████████████████████| 4.0MB 51.5MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 2.1MB 50.9MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 1.3MB 43.1MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 2.7MB 49.5MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
     |████████████████████████████████| 36.9MB 81kB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     

In [1]:
import pandas as pd

import syft as sy  # <-- import the Pysyft library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

torch.manual_seed(42)

pd.set_option("precision", 4)

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print("%d GPU(s) available, namely %s" % (torch.cuda.device_count(), torch.cuda.get_device_name(0)))
else:
    device = torch.device("cpu")
    print('No GPU available, using CPU instead.')


1 GPU(s) available, namely Tesla P100-PCIE-16GB


In [3]:
hook = sy.TorchHook(torch)  # <-- hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- and alice
print("hook: %s" % str(hook))

hook: <syft.frameworks.torch.hook.hook.TorchHook object at 0x7fd9cfe57390>


In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.log_interval = 10
        self.save_model = False

args = Arguments()

In [5]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST(
        '../data', train=True, download=True,
        transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.1307,), (0.3081,))
                                      ]))\
    .federate(
        (bob, alice)), # <-- we distribute the dataset across all the workers, it's now a FederatedDataset
        batch_size=args.batch_size, shuffle=True
        )

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        '../data', train=False, 
        transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.1307,), (0.3081,))
                                      ])),
                                      batch_size=args.test_batch_size, shuffle=True
                                      )

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!






In [10]:
# CNN Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(1, 32, 3, padding=0)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=0)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(p=0.25)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(12 * 12 * 64, 128)
        self.output = nn.Linear(128, 10)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = F.log_softmax(self.output(x), dim=1)
        return x
    

model = Net()
# model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)

In [11]:
"""
Train on the training data. Evaluate on the test data
"""
def train(args, model, device, train_loader, optimizer, epoch):

    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- send the model to the right location
        #data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- get the model back
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return
            

def evaluate(args, model, device, test_loader):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            #data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return

In [ ]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    evaluate(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")


Train Epoch: 1 [0/60032 (0%)]	Loss: 2.336147
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.234325
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.090510
Train Epoch: 1 [1920/60032 (3%)]	Loss: 1.920974
Train Epoch: 1 [2560/60032 (4%)]	Loss: 1.721464
Train Epoch: 1 [3200/60032 (5%)]	Loss: 1.466729
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.082324
Train Epoch: 1 [4480/60032 (7%)]	Loss: 0.997951
Train Epoch: 1 [5120/60032 (9%)]	Loss: 0.895159
Train Epoch: 1 [5760/60032 (10%)]	Loss: 0.904392
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.875989
Train Epoch: 1 [7040/60032 (12%)]	Loss: 0.693587
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.654471
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.785324
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.599998
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.650031
Train Epoch: 1 [10240/60032 (17%)]	Loss: 0.516612
Train Epoch: 1 [10880/60032 (18%)]	Loss: 0.718090
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.490374
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.691554
Train Epoch: 1 [12800/60032 (